# Paramter and base dev data

In [1]:
import sqlite3
import pandas as pd
import json

# =============================================================================
# Load configuration
# =============================================================================
with open("config.json", "r") as config_file:
    config = json.load(config_file)
DB_PATH = config["db_path"]

# =============================================================================
# CONFIGURE TARGET PREPARATION PARAMETERS
# =============================================================================
TABLE_NAME     = "bchusdt_1m"        # Define the table name here
OPEN_TIME_FROM = "2025-10-20 00:00"  # Define the start of the interval
OPEN_TIME_TO   = "2025-10-21 00:00"  # Define the end of the interval
ROLLING_WINDOW = 240                 # Rolling window in minutes (4 hours)

target = 'spike_flag'

# =============================================================================
# CONFIGURE INDICATORS AND THEIR PARAMETERS
# =============================================================================
# Define a structure for indicators and their parameters
indicator_config = {
    "momentum": {
        "rsi": [{"name": "rsi_backward", "window": 14}],
        "roc": [{"name": "roc", "window": 14}, {"name": "roc", "window": 140}]
    },
    "trend": {
        "macd": [{"name": "macd_diff"}],
        "sma": [
            {"name": "sma_ratio", "window": 14},
            {"name": "sma_ratio", "window": 140}
        ]
    },
    "volatility": {
        "bollinger_band": [
            {"name": "bb_width", "window": 14},
            {"name": "bb_width", "window": 140}
        ]
    }
}

# Generate the features list dynamically from indicator_config
features = []
for category, indicators in indicator_config.items():
    for indicator, params_list in indicators.items():
        for params in params_list:
            if "window" in params:
                features.append(f"{params['name']}_{params['window']}")
            else:
                features.append(params["name"])

print("Features list:", features)

# =============================================================================
# Fetch data from database
# =============================================================================
conn = sqlite3.connect(DB_PATH)
# SQL query to fetch data within the specified interval
query = f"""
    SELECT t.open_time, t.close
    FROM {TABLE_NAME} t
    WHERE open_time BETWEEN ? AND ?
"""
df = pd.read_sql_query(query, conn, params=(OPEN_TIME_FROM, OPEN_TIME_TO))

# Close the database connection
conn.close()
display(df.tail())

Features list: ['rsi_backward_14', 'roc_14', 'roc_140', 'macd_diff', 'sma_ratio_14', 'sma_ratio_140', 'bb_width_14', 'bb_width_140']


,open_time,close
1436,2025-10-20 23:56,481.5
1437,2025-10-20 23:57,481.1
1438,2025-10-20 23:58,481.4
1439,2025-10-20 23:59,481.4
1440,2025-10-21 00:00,480.9


# target

In [2]:
# =============================================================================
# Calculate rolling max, ratio, and 90th percentile target (spike_flag)
# =============================================================================

# Calculate the forward-looking rolling maximum
df["rolling_max"] = (
    df[::-1]  # Reverse the DataFrame to make the rolling operation forward-looking
    .rolling(window = ROLLING_WINDOW, min_periods = 1, on = "open_time")["close"]
    .max()
    .iloc[::-1]  # Restore the original order
    .reset_index(drop=True)
)

# Calculate the ratio of rolling maximum to the current close price
df["ratio"] = df["rolling_max"] / df["close"]

# Calculate the 90th percentile of the ratio
percentile_90 = df["ratio"].quantile(0.9)

# Add the spike_flag column based on the 90th percentile
df[target] = (df["ratio"] >= percentile_90).astype(int)

# =============================================================================
# Display the final DataFrame
# =============================================================================
display(df.tail())

,open_time,close,rolling_max,ratio,spike_flag
1436,2025-10-20 23:56,481.5,481.5,1.000000,0
1437,2025-10-20 23:57,481.1,481.4,1.000624,0
1438,2025-10-20 23:58,481.4,481.4,1.000000,0
1439,2025-10-20 23:59,481.4,481.4,1.000000,0
1440,2025-10-21 00:00,480.9,480.9,1.000000,0


# variables

In [3]:
import ta

# =============================================================================
# CALCULATE INDICATORS
# =============================================================================
# Momentum indicators
for params in indicator_config["momentum"]["rsi"]:
    df[f"{params['name']}_{params['window']}"] = ta.momentum.RSIIndicator(
        close=df["close"], window=params["window"]
    ).rsi()

for params in indicator_config["momentum"]["roc"]:
    df[f"{params['name']}_{params['window']}"] = ta.momentum.ROCIndicator(
        close=df["close"], window=params["window"]
    ).roc()

# Trend indicators
for params in indicator_config["trend"]["macd"]:
    macd_calc = ta.trend.MACD(close=df["close"])
    df[params["name"]] = macd_calc.macd_diff()

for params in indicator_config["trend"]["sma"]:
    df[f"{params['name']}_{params['window']}"] = df["close"] / ta.trend.SMAIndicator(
        close=df["close"], window=params["window"]
    ).sma_indicator()

# Volatility indicators
for params in indicator_config["volatility"]["bollinger_band"]:
    bb_calc = ta.volatility.BollingerBands(close=df["close"], window=params["window"])
    df[f"{params['name']}_{params['window']}"] = bb_calc.bollinger_wband()

# =============================================================================
# FINAL FEATURES + TARGET AND MISSING ROWS
# =============================================================================
df.tail()

,open_time,close,rolling_max,ratio,spike_flag,rsi_backward_14,roc_14,roc_140,macd_diff,sma_ratio_14,sma_ratio_140,bb_width_14,bb_width_140
1436,2025-10-20 23:56,481.5,481.5,1.000000,0,44.366243,-0.041520,-0.475403,0.114712,1.000460,0.995594,0.361156,1.013629
1437,2025-10-20 23:57,481.1,481.4,1.000624,0,37.910544,-0.145289,-0.578632,0.095620,0.999733,0.994808,0.341891,1.028687
1438,2025-10-20 23:58,481.4,481.4,1.000000,0,44.440293,-0.124481,-0.557736,0.101909,1.000445,0.995468,0.296172,1.039586
1439,2025-10-20 23:59,481.4,481.4,1.000000,0,44.440293,-0.062279,-0.598802,0.104398,1.000490,0.995511,0.276801,1.049550
1440,2025-10-21 00:00,480.9,480.9,1.000000,0,36.932598,-0.083108,-0.783990,0.072229,0.999510,0.994533,0.280362,1.062923


# model

In [ ]:
import statsmodels.api as sm

# =============================================================================
# CREATE df_sample BY FILTERING df
# =============================================================================
# Drop rows with NA in any explanatory variable
df_sample = df.dropna()

# Ensure the 'open_time' column is in datetime format using .loc
df_sample.loc[:, "open_time"] = pd.to_datetime(df_sample["open_time"])

# Exclude the most recent 240 minutes of observations
cutoff_time = df_sample["open_time"].max() - pd.Timedelta(minutes=ROLLING_WINDOW)
df_sample   = df_sample[df_sample["open_time"] < cutoff_time]

# =============================================================================
# LOGISTIC REGRESSION MODEL (sm)
# =============================================================================
# Define explanatory variables (X) and target variable (y)
X = df_sample[features]
y = df_sample[target]

# Fit the logistic regression model
logit_model = sm.Logit(y, X)
result = logit_model.fit()

# Display the second table (tables[1]) from the summary report
display(result.summary().tables[1])

Optimization terminated successfully.
         Current function value: 0.369084
         Iterations 8


,coef,std err,z,P>|z|,[0.025,0.975]
rsi_backward_14,-0.0072,0.016,-0.438,0.661,-0.039,0.025
roc_14,-1.1961,0.779,-1.536,0.125,-2.722,0.330
roc_140,-1.2750,0.275,-4.640,0.000,-1.813,-0.736
macd_diff,3.3175,1.651,2.010,0.044,0.082,6.553
sma_ratio_14,-154.4463,61.343,-2.518,0.012,-274.677,-34.215
sma_ratio_140,152.9291,61.434,2.489,0.013,32.520,273.338
bb_width_14,1.1825,0.315,3.756,0.000,0.565,1.800
bb_width_140,-0.4395,0.198,-2.219,0.027,-0.828,-0.051
